In [156]:
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
from opmd_viewer import OpenPMDTimeSeries

In [ ]:
!rm -r /mnt/hdd/data/gpu_run_debug0/*
!/home/dominik/.miniconda3/envs/nbody3.6/bin/python /home/dominik/Code/NBody-MD/nbody/run_nbody.py --config /home/dominik/Code/NBody-MD/config.json

/home/dominik/.miniconda3/envs/nbody3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 88 from C header, got 96 from PyObject
  return f(*args, **kwds)
 51%|▌| 2568/5000 [07:24<06:47,  5.97it/s, kinetic_energy=2.75e-18, mean_p=(-1.1515310260435617e-39, 4.0673080290597826e-39, -9.602549067019956e-39), mean_r=(1.5279999999999982e-09, 1.5280000000000018e-09, 1.5280000000000046e-09), potential_energy=1.42e-15, std_p=(1.5733264131521722e-23, 1.5815909241772493e-23, 1.465418707214407e-23), std_r=(8.780119353566185e-10, 8.7900985722298e-10, 8.753195867705408e-10), temperature=260]

In [151]:
ts = OpenPMDTimeSeries('/mnt/hdd/data/gpu_run_debug0')
ts.slider()

FigureCanvasNbAgg()

In [152]:
# ts.h5_files

# import h5py
# h5_files = [h5py.File(f) for f in ts.h5_files[:3]]

# for f in h5_files:
#     f.close()

In [153]:
import pandas as pd
df = pd.read_json('/mnt/hdd/data/gpu_run_debug0/diagnostic_results.json').T
df["total_energy"] = df["kinetic_energy"] + df["potential_energy"]

from IPython.display import display

unwrap = ['mean_r', 'std_r', 'mean_p', 'std_p']
nested_data = {index: pd.DataFrame(df[index].values.tolist(), index=df.index, columns=['x', 'y', 'z']) for index in unwrap}
directional_values = pd.concat(nested_data).stack().unstack(level=1).T
# directional_values

directional_values['mean_p'].plot()

FigureCanvasNbAgg()

In [154]:
df.drop(columns=unwrap).drop(columns=['temperature']).plot()

FigureCanvasNbAgg()

In [155]:
df.plot(y='temperature')

FigureCanvasNbAgg()